# Deploy to Azure Container Instance

In this section, we will take the pretrained model from the previous notebook and deploy it to a web service using Azure Container Instance.

This example shows how to deploy a web service in step-by-step fashion:

- Register model
- Create Docker image
- Deploy the image as web service

In [20]:
import sys
import os
import shutil
import numpy as np

from reco_utils.dataset import movielens

import azureml
from azureml.core import Workspace, Run
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

### Connect to an AzureML workspace

An [AzureML Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows. In particular, an Azure ML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, deployment, inferencing, and the monitoring of deployed models.

A workspace has already been created for you and a [configuration file](././home/nbuser/library/config.json). Simply load it with ```ws = Workspace.from_config()```. You may be asked to login. Please follow the prompts to 

**More information on creating your own workspace can be found in this [this tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace#portal).

In [3]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: /home/nbuser/library/config.json
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FQNXHQ56U to authenticate.
Interactive authentication successfully completed.


### Reminder of the data
In the previous notebook, we looked at the Movielens dataset. The dataset contains users' rankings for different movies. You can see a sample below.

In [4]:
# download dataset
data = movielens.load_pandas_df(
    size='100k',
    header=['UserId','MovieId','Rating','Timestamp'],
    title_col='Title'
)
data.head()

4.93MB [00:01, 4.11MB/s]                            


,UserId,MovieId,Rating,Timestamp,Title
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


### Register the model 
You can add tags and descriptions to your models. Note you need to have a `movielens_sar_model.pkl` file in the current directory. This file is generated by the `sar_movielens_with_azureml` notebook. The below call registers that file as a model with the name `movielens_sar_model` in the workspace.

**Note: You can change the name of the model, but you will need to update it in the score.py script below. Models are versioned. If you call the register command many times with same model name, you will get multiple versions of the model with increasing version numbers. **

In [29]:
model = Model.register(ws,model_name = 'movielens_sar_model',
                       model_path = './movielens_sar_model.pkl',
                       description ="Building a movie recommendation system")
print(model.name, model.id, model.version, sep = '\t')

Registering model movielens_sar_model
movielens_sar_model	movielens_sar_model:12	12


### Create Docker Image
Below we create a scoring script that the web service will use to predict new data. Note that the `movielens_sar_model` in the get_model_path call is referring to a model named ``movielens_sar_model`` registered under the workspace. It is NOT referencing the local file.

In [14]:
%%writefile score.py

import json
import numpy
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.externals import joblib
from azureml.core.model import Model
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

# load the model
def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path(model_name='movielens_sar_model')
    model = joblib.load(model_path)

# Passes data to the model and returns the prediction
def run(raw_data):
    # make prediction
    try: 
        data = raw_data
        data = pd.read_json(data)
        return model.get_item_based_topk(items=data, sort_top_k=True).to_json()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [30]:
myenv = CondaDependencies()
myenv.add_conda_package("numpy")
myenv.add_conda_package("pandas")
myenv.add_conda_package("tqdm")
myenv.add_pip_package('sklearn')

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [17]:
# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                 runtime = "python",
                                                 conda_file = "myenv.yml",
                                                 dependencies = ["reco_utils"]
                                                 )

**Note that the following command can take a few minutes to run.**

In [18]:
# Register the image from the image configuration
image = ContainerImage.create(name = "myimage",
                              models = [model], #this is the model object
                              image_config = image_config,
                              workspace = ws
                              )
image.wait_for_creation(show_output = True)

Creating image
Running..................................................
SucceededImage creation operation finished for image myimage:29, operation "Succeeded"


### Deploy image as web service on Azure Container Instance
**Note that service creation can take a few minutes to run.**

In [19]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'movielens')

In [22]:
service_name = 'simlar-movielens-test2'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)
print(service.state)

Creating service
Running.......................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


### Test out the brand new webservice 
Call the web service with the below widget to get a recommendation of similar movies.

In [27]:
%run widget.ipynb

Label(value='Here are the recommended movies based on your ratings.', style=DescriptionStyle(description_width…

,MovieId,UserId,prediction,Title
0,50,0,4.615843,Star Wars (1977)
1,174,0,4.486288,Raiders of the Lost Ark (1981)
2,172,0,4.480662,"Empire Strikes Back, The (1980)"
3,98,0,4.413595,"Silence of the Lambs, The (1991)"
4,181,0,4.365086,Return of the Jedi (1983)
5,1,0,4.324029,Toy Story (1995)
6,56,0,4.230594,Pulp Fiction (1994)
7,96,0,4.148080,Terminator 2: Judgment Day (1991)
8,121,0,4.147672,Independence Day (ID4) (1996)
9,168,0,4.124218,Monty Python and the Holy Grail (1974)


### Delete ACI to clean up
If you want to clean up your workspace and delete the service, you can uncomment the following line of code. 

In [ ]:
# aci_service.delete()